# About this Page
This section is a developer aid; TODO remove later!
*This notebook is intended to align with [storyboard Version 51](https://rubinobs.atlassian.net/wiki/pages/viewpage.action?pageId=132612364&pageVersion=51)* (more or less)
If you are using this under Time Square and *get any errors* that you think come from this notebook, please tell Steve Pothier (steve.pothier@noirlab.edu).  Include the full Times Square URL you used to get the error. 

## What is new?
- Added "What is new"
- Retrieve of records from each source will repeatedly get pages until all data for the specified date range has been retrieved.
- Data Log partitions exposure counts by "quality" flags.
- Data Log now has link to drill down to Detail exposure list.
- This page tries to work around existing problems with APIs used for source data. (There is a ticket to fix the APIs)
- Errors with APIs are display in this page in Red.

## Known issues
- Sunset bug: DM-47511

In [1]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "YESTERDAY"  # TODO Change to 'YESTERDAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "1"  # TODO Change to '1' to test with default before push

verbose = "true"  # TODO change to false before push, else true

In [2]:
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode, urlparse
import pandas as pd
from IPython.display import HTML

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        'TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP.'
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    dev_msg = 'Imported lsst.ts.logging_and_reporting from local packages.'
from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_reports import AllReports
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md, mdlist, mdpathlink
from lsst.ts.logging_and_reporting.reports import html_draft, html_beta

In [3]:
# Set default env to "usdf" and try before PUSH to repo.
# The default provided here is for local testing. Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", 
    ut.Server.usdf) # TODO try with "usdf" before push (else "summit")

In [4]:
date = ut.get_datetime_from_dayobs_str(day_obs)
# date:  is EXCLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

verbose = (verbose == "true") 


In [5]:
# Read records from (almost) all sources
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    verbose=verbose,                
    limit=5000,
    exclude_instruments = []      # TODO change to empty list before push
)
allrep = AllReports(allsrc=allsrc)

DBG get_records endpoint='https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports'
DBG get_records qstr: is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241111&max_day_obs=20241112
DEBUG protected_get(url='https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241111&max_day_obs=20241112',timeout=(5.05, 20.0))
DBG get_records-2 len(page)=2 len(recs)=2
DBG get_records-2 len(self.records)=2 status={'endpoint_url': 'https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241111&max_day_obs=20241112', 'number_of_records': 2, 'error': None}
DEBUG protected_get(url='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/instruments',timeout=(5.05, 20.0))
DBG get_exposures endpoint='https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures'
DBG get_exposures qstr: registry=1&instrument=LSSTCo

----------

In [6]:
instrum_str = ", ".join(list(allsrc.exp_src.instruments.keys()))
md(f"# Showing data for {min_date.date()} to {max_date.date()} for {instrum_str}")

# Showing data for 2024-11-11 to 2024-11-12 for LSSTComCamSim, LATISS, LSSTComCam, LSSTCam

# Table of Contents
* [Night Report](#Night-Report-BETA)
    - AuxTel
    - Simonyi
* [Almanac](#Almanac-BETA)
* [Summary plots of whole night](#Summary-plots-of-whole-night-DRAFT)
* [Time Accounting](#Time-Accounting-DRAFT)
* [Jira Tickets](#Jira-Tickets-BETA)
    - AuxTel
    - Simonyi
* [BLOCKS Observed](#BLOCKS-Observed-DRAFT)
    - AuxTel
    - Simonyi
* [Data Log](#Data-Log-DRAFT)
* [Narrative Log](#Narrative-Log-BETA)
* [Developer Only Section](#Developer-Only-Section-REMOVE)

## Night Report <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [7]:
# Night Report
# Display time log
allrep.nig_rep.time_log_as_markdown()

### Telescope: Simonyi

```
Site was running on generator #2 upon arrival.  Day shift started with powering on and warming up M2 and Camera hexapods.  During the bump test, the load was switched to the Caterpillar generator, which apparently glitched the air supply to M1M3, so several actuators reported failed.  These were re-tested and cleared by Petr.  FA 215 was known to be bad and was disabled.  The TMA and Dome both presented trouble during their respective handovers (OBS-662 for TMA, OBS-651 for Dome).  Two laser tracker alignments were attempted but the first was not useful since the rotator was not at zero, and the second run was successful.  Despite the successful earlier dome handover, we had to recover communication wit the dome through sending exitFault commands and state cycling MTDome until we recovered telemetry and control.  Telemetry rate is quite slow, sometimes taking ~30 seconds to return.  At 8:30pm local time, the temperature conditions in and around M1M3 and outside were matched within <2 degrees, and the shutters were opened.

On-sky observations kicked off with twilight flats, and a full set of useful r-band flats were acquired.  

------- AOS Night Summary -------

Tonight was productive, with significant learning and progress despite challenging seeing conditions. The start of the night was  disrupted by dome and mount issues, as well as issues with the closed-loop script returning NaNs. After completing a few i and r filter sequences for pipeline validation, we moved on to our main goal: refining our understanding of misalignments.

Building on the best M2 ry tilt from the previous night, we systematically perturbed each hexapod degree of freedom, closed the loop, and checked for any sign or rotation errors. We addressed the remaining issues, achieving a successful repeat of the experiment with the closed loop correcting accurately in the opposite direction of each applied perturbation. We then closed the loop using M2 rx and ry tilts to refine telescope alignment, converging on an M2 ry tilt of about 0.03 degrees, with M2 rx at zero. This result aligns better with expectations than our earlier Cam dx value of 4.8 mm, and we'll keep this as our baseline.

Following this, we began closing the loop on z and tilts, then introduced M1M3 bending modes. While running the closed-loop script, we noted that sometimes corrections were not being applied due to our tolerance threshold being too larges—an important tweak for future sessions. Poor seeing prevented us from achieving sub-arcsecond image quality until the end of the night (we got to 0.96 arcsec in injury time!), but we reached seeing-limited quality, a solid outcome given the conditions.

Although the root cause of the sign flips still requires tracing, we now have a reliable algorithm for returning to focus. In the coming nights, we'll aim to close the loop with multiple degrees of freedom, fine-tuning the loop and addressing degeneracies. Updating the elevation LUT is also a priority, as frequent refocusing after elevation changes has become cumbersome. Work needs to be done to increase the robustness of the AOS to variations in sky background. We'll leverage data from test T222 for improvements. And, whoever is up there with a hair dryer affecting our seeing, kindly knock it off!

At 05:06 local we started the TMA/Dome parking procedures, closed dome calibration images and the end of night procedures. MTRotator refused to move to 0 deg, only after a Rot-PXI reboot we were able to do it (OBS-664). Once TMA was parked at horizon, all the HP forces were below +/-1800N and while lowering the mirror faulted in breakaway. MTM1M3TS was cycled with fans running. ComCam calibrations on the go while going down to the hotel.
```

Confluence page: [v2/testCases](https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/v2/testCases)

Telescope Status: TMA at Horizon, mirror covers retracted. TMA Az=150.8 deg. Dome at Az=80 deg.
MTM1M3 DISABLED - lowered and disabled
MTM1M3TS ENABLED
MTM2 ENABLED in closed loop
MTAOS STANDBY
MTDome DISABLED
MTDomeTrajectory STANDBY
MTMount DISABLED
MTHexapods in STANDBY w/ cabinets and PXIs powered off
MTPtg ENABLED
MTRotator at 0º, in STANDBY w/ cabinets and PXIs powered off
CCCamera, CCOODS, CCHeaderService in ENABLED

*Authors: Eric Christensen, David Sanmartim, Paulina Venegas, Karla Pena, Elana Urbach, Guillem Megias Homar, Andrew Connolly*

```
Site was running on generator #2 upon arrival.  Day shift started with powering on and warming up M2 and Camera hexapods.  During the bump test, the load was switched to the Caterpillar generator, which apparently glitched the air supply to M1M3, so several actuators reported failed.  These were re-tested and cleared by Petr.  FA 215 was known to be bad and was disabled.  The TMA and Dome both presented trouble during their respective handovers (OBS-662 for TMA, OBS-651 for Dome).  Two laser tracker alignments were attempted but the first was not useful since the rotator was not at zero, and the second run was successful.  Despite the successful earlier dome handover, we had to recover communication wit the dome through sending exitFault commands and state cycling MTDome until we recovered telemetry and control.  Telemetry rate is quite slow, sometimes taking ~30 seconds to return.  At 8:30pm local time, the temperature conditions in and around M1M3 and outside were matched within <2 degrees, and the shutters were opened.

On-sky observations kicked off with twilight flats, and a full set of useful r-band flats were acquired.  

------- AOS Night Summary -------

Tonight was productive, with significant learning and progress despite challenging seeing conditions. The start of the night was  disrupted by dome and mount issues, as well as issues with the closed-loop script returning NaNs. After completing a few i and r filter sequences for pipeline validation, we moved on to our main goal: refining our understanding of misalignments.

Building on the best M2 ry tilt from the previous night, we systematically perturbed each hexapod degree of freedom, closed the loop, and checked for any sign or rotation errors. We addressed the remaining issues, achieving a successful repeat of the experiment with the closed loop correcting accurately in the opposite direction of each applied perturbation. We then closed the loop using M2 rx and ry tilts to refine telescope alignment, converging on an M2 ry tilt of about 0.03 degrees, with M2 rx at zero. This result aligns better with expectations than our earlier Cam dx value of 4.8 mm, and we'll keep this as our baseline.

Following this, we began closing the loop on z and tilts, then introduced M1M3 bending modes. While running the closed-loop script, we noted that sometimes corrections were not being applied due to our tolerance threshold being too larges—an important tweak for future sessions. Poor seeing prevented us from achieving sub-arcsecond image quality tonight, but we reached seeing-limited quality, a solid outcome given the conditions.

Although the root cause of the sign flips still requires tracing, we now have a reliable algorithm for returning to focus. In the coming nights, we'll aim to close the loop with multiple degrees of freedom, fine-tuning the loop and addressing degeneracies. Updating the elevation LUT is also a priority, as frequent refocusing after elevation changes has become cumbersome. Work needs to be done to increase the robustness of the AOS to variations in sky background. We'll leverage data from test T222 for improvements. And, whoever is up there with a hair dryer affecting our seeing, kindly knock it off!

At 05:06 local we started the TMA/Dome parking procedures, closed dome calibration images and the end of night procedures. Once TMA was parked at horizon, all the HP forces were below +/-1800N and while lowering the mirror faulted in breakaway. MTM1M3TS was cycled with fans running.
```

Confluence page: [v2/testCases](https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/v2/testCases)

Telescope Status: TMA at Horizon, mirror covers retracted. TMA Az=150.8 deg. Dome at Az=80 deg.
MTM1M3 DISABLED - lowered and disabled
MTM1M3TS ENABLED
MTM2 ENABLED in closed loop
MTAOS STANDBY
MTDome DISABLED
MTDomeTrajectory STANDBY
MTMount DISABLED
MTHexapods in STANDBY w/ cabinets and PXIs powered off
MTPtg ENABLED
MTRotator at 0º, in STANDBY w/ cabinets and PXIs powered off
CCCamera, CCOODS, CCHeaderService in ENABLED

*Authors: Eric Christensen, David Sanmartim, Paulina Venegas, Karla Pena, Elana Urbach, Guillem Megias Homar, Andrew Connolly*

## Almanac <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [8]:
# Display various almanac values (for moon, sun)
#!rep.AlmanacReport().day_obs_report(min_day_obs)
allrep.alm_rep.day_obs_report()

**Almanac for the observing night starting 2024-11-11**

,(times in UTC),
Moon Rise,2024-11-10 17:43:00.925,
Moon Set,2024-11-11 06:41:58.146,
Moon Illumination,71%,(% illuminated)
Astronomical Twilight (morning),2024-11-11 08:10:48.514,(-18 degrees)
Astronomical Twilight (evening),2024-11-11 00:42:56.117,(-18 degrees)
Solar Midnight,2024-11-11 04:26:52.295,
Nautical Twilight (morning),2024-11-11 08:43:18.209,(-12 degrees)
Nautical Twilight (evening),2024-11-11 00:10:26.410,(-12 degrees)
Civil Twilight (morning),2024-11-11 09:14:21.799,(-6 degrees)
Civil Twilight (evening),2024-11-10 23:39:22.778,(-6 degrees)


## Summary plots of whole night <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

(content not yet defined in storyboard)

In [9]:
# Plot Observation (Exposure) gaps
# rollup,detail = allsrc.get_observation_gaps()
# allrep.plot_observation_gap_detail(detail)

## DDV <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

In [10]:
DDV = (
    f"{server}/rubintv-dev/ddv/index.html"
    if "summit" in server
    else f"{server}/rubintv/ddv/index.html"
)
md(f"Access DDV part of RubinTV: {DDV}")

Access DDV part of RubinTV: https://usdf-rsp-dev.slac.stanford.edu/rubintv/ddv/index.html

## Time Accounting <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
TODO:
- Time-loss for Fault and Weather
- SlewTime from TMAEvent (or similar)

SlewTime (and probably others) from EFD are only EXPECTED (topic="lsst.sal.Scheduler.logevent_target"), not ACTUAL.
To get ACTUAL time, we need to use something like TMAEvent to combine multiple real events into a synthetic event.  See DM-46102.


In [11]:
# Exposure Tally
tally = await allsrc.night_tally_observation_gaps()
if tally:
    display(pd.DataFrame(tally))

,LSSTComCam
Total Night,7:27:52
Total Exposure,2:14:18
Slew time(1),0:00:00
Readout time(2),0:12:55
Time loss to fault,NA
Time loss to weather,NA
Idle time,5:13:33
Number of exposures,337
Mean readout time,0.000639
Number of slews(1),0


- (1) *Expected* slew time as per Schedular
- (2) There is no practical way to get detector read-out time.  A value of 2.3 seconds was inferred from experiments.

## Jira Tickets <font style="background-color:green; color:white; font-size:20px">BETA</font> 
- Observing Operations - OBS
- TODO: better tickets using Jira API

In [12]:
if allsrc.urls:
    for url in allsrc.urls:
        md(f"- {mdpathlink(url)}")

- [OBS-662](https://rubinobs.atlassian.net/browse/OBS-662)

- [OBS-614](https://rubinobs.atlassian.net/browse/OBS-614)

- [OBS-659](https://rubinobs.atlassian.net/browse/OBS-659)

- [OBS-651](https://rubinobs.atlassian.net/browse/OBS-651)

In [13]:
# Display Jira BLOCKS
front = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/"
tickets = allsrc.nig_src.nightly_tickets()
if tickets:
    mdstr = "#### Nightly Jira BLOCKs"
    for day, url_list in tickets.items():
        mdstr += f"\n- {day}"
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    endpoint_url = allsrc.nig_src.status["reports"]["endpoint_url"]
    md(f"No tickets found using: [API Data]({endpoint_url}) in `confluence_url`.")

#### Nightly Jira BLOCKs
- 20241111
    - [v2/testCases](https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/v2/testCases)

## BLOCKS Observed <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
- TODO: BLOCKS observed

## Data Log <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

- Should we replace zero with empty string?
- TODO: Add link to exposures detail when A field value (e.g. observation_reason=science) is clicked on.  The detail should include all exposures counted int the total for that field value.

In [14]:
# Exposure Report
for instrum in allsrc.exp_src.exposures.keys():
    md(f'**{instrum}: {len(allsrc.exp_src.exposures[instrum])} total exposures**')
    df_dict = allsrc.fields_count_exposure(instrum)
    for field_name,df in df_dict.items():
        if df.empty:
            continue
        print(field_name.title().replace('_',' '))
        display(HTML(df.style.hide().to_html(escape=False)))

**LSSTComCamSim: 0 total exposures**

**LATISS: 0 total exposures**

**LSSTComCam: 337 total exposures**

Observation Type


Detail,good,questionable,junk,unknown
acq,0,0,0,123
bias,0,0,0,37
cwfs,0,0,0,124
dark,0,0,0,27
focus,0,0,0,5
science,0,0,0,21


Observation Reason


Detail,good,questionable,junk,unknown
acq,0,0,0,29
bias,0,0,0,5
comcam-daily-calibration,0,0,0,44
comcam-usdf-checkout,0,0,0,10
comcam_twilight_flat,0,0,0,38
extra,0,0,0,2
extra_repeteability_wavefront,0,0,0,11
extra_sitcom-826,0,0,0,48
focus_sweep_cam_dz,0,0,0,5
infocus,0,0,0,2


Science Program


Detail,good,questionable,junk,unknown
BLOCK-T215,0,0,0,38
BLOCK-T220,0,0,0,44
BLOCK-T235,0,0,0,14
BLOCK-T246,0,0,0,10
BLOCK-T248,0,0,0,2
BLOCK-T249,0,0,0,10
BLOCK-T278,0,0,0,41
BLOCK-T279,0,0,0,19
BLOCK-T280,0,0,0,4
BLOCK-T92,0,0,0,5


**LSSTCam: 0 total exposures**

## Narrative Log <font style="background-color:green; color:white; font-size:20px">BETA</font> 

In [15]:
# Narrative Report
# Time Log
allrep.nar_rep.time_log_as_markdown()

- **2024-11-11 04:18:44.671682** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>MTDome telemetry returning to normal</pre>


- **2024-11-11 04:17:45.689723** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Tried move_rotator script, but failed. Can't move the rotator to enabled mode. 
The difference between the rotator and CCW positions is out of range to enable CSC.   


Alternative procedure: move CCW to rotator position through the EUI 
	Put MTRotator and MTMount in Disabled 
	Take control in the EUI 
	CCW, reset alarm, turn on, select 1, input the rotator position of 5.8 deg, move, reset alarm, turn off 
	Main axes, reset alarm


Send MTMount to Enabled 
Then  MTForceError.MTM2 fault
	 (the range is not tuned for the AOS tests)</pre>


- **2024-11-11 04:09:33.289017** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Rotator position is 5.874 and the CCW position is 3.674.</pre>


- **2024-11-11 04:06:39.542003** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>MTPtg wen to fault


 Error Rotator replied error for trackTarget command (1735273186) Error no:1 message:Failed: Must be in Enabled state
0</pre>


- **2024-11-11 04:06:06.989743** Time Lost: 0.0; Time Lost Type: fault;
    - components: None
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>MTRotator wen to fault


Camera cable wrap not following closely enough: error # 3 = 2.2011443473732264 &gt; 2.2 deg</pre>


...

- **2024-11-11 14:44:02.085067** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Visual inspection with FA215 looked good, Christian and Roberto opened the FA215's valve.</pre>


- **2024-11-11 13:24:52.800785** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>LOTO on, Christian gets into the M1M3 cell to inspect FA 215</pre>


- **2024-11-11 13:10:27.057259** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>TMA parked at zenith, az=-31. locking pin in. PS off, OSS off.</pre>


- **2024-11-11 13:08:09.819671** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>MTPtg ENABLED, MTMount DISABLED</pre>


- **2024-11-11 12:58:37.079807** Time Lost: 0.0; Time Lost Type: fault;
    - components: ['MainTel']
    - primary_software_components: ['None']
    - primary_hardware_components: ['None']

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Day crew came. Need TMA zenith and az=-31 for access to the M1M3 cell.
PS on, switched to the Parking set, David Jimenez moving TMA in EUI.</pre>


-----------
------------

# Developer Only Section <font style="background-color:red; olor:black; font-size:20px">REMOVE</font>
Contains stuff only expected to be useful to developers.

This may also contain sections that have moved out of the user section because they are no longer defined in the Storyboard.

## Overview 


In [16]:
# Display overview of Report context
try:
    import lsst.ts.logging_and_reporting.version

    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient

    have_consdb = True
except:
    have_consdb = False
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- min_dayobs={allsrc.min_dayobs!r}, max_dayobs={allsrc.max_dayobs!r}
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
- {have_consdb = }
- {dev_msg}
- {allsrc.nig_src.timeout=}
- {allsrc.nar_src.timeout=}
""")

## Project-Wide Night(s) Report 
- Run on logs and databases from **https://usdf-rsp-dev.slac.stanford.edu/**
- Report **1 observing night(s)** with the last reported night starting on **2024-11-11 12:00:00**.
- min_dayobs='2024-11-11', max_dayobs='2024-11-12'
- Using ***Prototype* Logging and Reporting** Version: **0.1.dev145+gf2a5765.d20241014**
- have_consdb = True
- Imported lsst.ts.logging_and_reporting from local packages.
- allsrc.nig_src.timeout=(5.05, 20.0)
- allsrc.nar_src.timeout=(5.05, 20.0)


## Data Status
This may be useful for diagnosing reports that give unexpected results.

In [17]:
df = pd.DataFrame(allsrc.get_data_status())
table = df.to_html(
    index=False, 
    render_links=True,
    escape=False)
display(HTML(table))

Endpoint,Records,URL
nightreport/reports,2,https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241111&max_day_obs=20241112
exposurelog/instruments,3,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/instruments
exposurelog/exposures.LSSTComCamSim,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCamSim&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241111&max_day_obs=20241112
exposurelog/exposures.LATISS,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LATISS&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241111&max_day_obs=20241112
exposurelog/exposures.LSSTComCam,337,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241111&max_day_obs=20241112
exposurelog/exposures.LSSTCam,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241111&max_day_obs=20241112
exposurelog/messages,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/messages?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=2500&min_day_obs=20241111&max_day_obs=20241112
narrativelog/messages,82,https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-11T00%3A00%3A00&max_date_added=2024-11-12T00%3A00%3A00
efd/targets,0,NA


## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Section overviews moved here

In [18]:
# Night Report Overview
allrep.nig_rep.overview()
# Exposure Report Overview
allrep.exp_rep.overview()
# Narrative Report Overview
allrep.nar_rep.overview()

### Overview for Service: `nightreport` [2]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241111&max_day_obs=20241112

None
- Got 2 records.  


### Overview for Service: `exposurelog` [0]

...

None
- Got 0 records.  


### Overview for Service: `narrativelog` [82]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-11T00%3A00%3A00&max_date_added=2024-11-12T00%3A00%3A00

None
- Got 82 records.  


In [19]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

### According to Storybaord, this section should be removed ...
... and replaced with a simple table tally of Good, Questionable, and  Junk exposures per instrument.
| Symbol | Meaning |
|:---|:---|
| <font style="background-color:green; color:white; font-size:20px">&nbsp;G&nbsp;</font>| Good |
| <font style="background-color:yellow; color:black; font-size:20px">&nbsp;?&nbsp;</font> | Questionable |
| <font style="background-color:red; color:black; font-size:20px">&nbsp;R&nbsp;</font> | Junk |


In [20]:
# Time Log
allrep.exp_rep.time_log_as_markdown()

<font style="color:lightblue;background-color:None;">No exposurelog records found 2024-11-11 to 2024-11-12. </font>

Used [API Data](https://usdf-rsp-dev.slac.stanford.edu/exposurelog/messages?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=2500&min_day_obs=20241111&max_day_obs=20241112)

## Finale

In [21]:
print(f"Finished {str(dt.datetime.now())}")

Finished 2024-11-12 23:40:10.647837
